In [9]:
from collections import Counter
from collections import OrderedDict
import numpy as np
import pandas

In [10]:
visit_counter = Counter()
purchase_counter = Counter()
with open('coursera_sessions_train.txt', 'r') as f:
    for l in f.readlines():
        visits, purchases = l.strip().split(';')
        for visit in visits.split(','):
            visit_counter[visit] += 1
        if purchases != '':
            for purchase in purchases.split(','):
                purchase_counter[purchase] += 1
                

In [11]:
def _purchase(items, max_count):
    return sorted(OrderedDict.fromkeys(items), key=lambda x: purchase_counter.get(x, 0), reverse=True)[:max_count]
def _visit(items, max_count):
    return sorted(OrderedDict.fromkeys(items), key=lambda x: visit_counter.get(x, 0), reverse=True)[:max_count]


In [12]:
def _metrics(recommend, max_count, file):
    with open(file, 'r') as f:
        _recall = np.zeros(max_count)
        _precision = np.zeros(max_count)
        _count = 0
        for l in f.readlines():
            visits, purchases = l.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = set(purchases.split(','))
                rec = recommend(visits, max_count)
                rec_hits = list(map(lambda x: x in purchases, rec))
                hits = np.zeros(max_count)
                hits[:len(rec_hits)] = rec_hits
                _count += 1
                _recall +=  np.cumsum(hits) / len(purchases)
                _precision += np.cumsum(hits) / (np.arange(max_count) + 1)

    return pandas.DataFrame({
            'recall': [round(x, 2) for x in _recall / _count],
            'precision': [round(x, 2) for x in _precision / _count],
            'k': np.arange(max_count) + 1,
    }).set_index('k')

In [13]:
_metrics(_purchase, 5, 'coursera_sessions_train.txt')

,recall,precision
k,,
1,0.69,0.80
2,0.84,0.53
3,0.89,0.39
4,0.91,0.31
5,0.93,0.25


In [14]:
_metrics(_purchase, 5, 'coursera_sessions_test.txt')

,recall,precision
k,,
1,0.46,0.53
2,0.64,0.38
3,0.73,0.30
4,0.79,0.25
5,0.82,0.21


In [15]:
_metrics(_visit, 5, 'coursera_sessions_test.txt')

,recall,precision
k,,
1,0.42,0.48
2,0.60,0.36
3,0.70,0.29
4,0.76,0.24
5,0.80,0.20


In [16]:
_metrics(_visit, 5, 'coursera_sessions_test.txt')

,recall,precision
k,,
1,0.42,0.48
2,0.60,0.36
3,0.70,0.29
4,0.76,0.24
5,0.80,0.20
